- Load two eval_results

EvalResult
- example, label, result, item_summary

In [ ]:
import typing
import weave
import random
import string
from weave import weave_internal
weave.use_frontend_devmode()
from weave.panels import panel_board
from weave import ops_domain

In [ ]:
def rand_string_n(n: int) -> str:
    return "".join(
        random.choice(string.ascii_uppercase + string.digits) for _ in range(n)
    )

dataset_raw = [{
    'id': str(i),
    'example': rand_string_n(10),
    'label': random.choice(string.ascii_uppercase)} for i in range(50)]
dataset = weave.save(dataset_raw, 'dataset')
#dataset

In [ ]:
def predict(dataset_row, config):
    if random.random() < config['correct_chance']:
        return dataset_row['label']
    return random.choice(string.ascii_uppercase)

In [ ]:
def evaluate(dataset, predict_config):
    eval_result = []
    correct_count = 0
    count = 0
    for dataset_row in dataset:
        result = predict(dataset_row, predict_config)
        correct = dataset_row['label'] == result
        if correct:
            correct_count += 1
        count +=1 
        eval_result.append({
            'dataset_id': dataset_row['id'],
            'result': result,
            'summary': correct
        })
    return {
        'config': predict_config,
        'eval_table': eval_result,
        'summary': {'accuracy': correct_count / len(dataset)}}

In [ ]:
eval_result_raw0 = evaluate(dataset_raw, {'correct_chance': 0.5})
eval_result_raw1 = evaluate(dataset_raw, {'correct_chance': 0.5})
eval_result0 = weave.save(eval_result_raw0, 'eval_result0')
eval_result1 = weave.save(eval_result_raw1, 'eval_result1')

In [ ]:
# TODO:
#   - works on 2, make work on N
#   - default table and panel behavior is horrible
#   - automatically make good columns
#   - make an example for text extraction (how is that usually evaluated?)
#   - colors all the way through
#   - controls?
#   - hard examples, confusing examples
#   - auto update when inputs change?
#   - or do you just publish a new board when that happens?
#   - maybe we want like an overview tab where we select a set of runs
#     with latency v accuracy or whatever
#     - and then an example drilldown tab where you just pick two and compare
#   - going to want to compare traces for the LLM extraction case


varbar = panel_board.varbar()

dataset_var = varbar.add('dataset', dataset)
eval_result0_var = varbar.add('eval_result0', eval_result0)
eval_result1_var = varbar.add('eval_result1', eval_result1)

summary = varbar.add('summary', weave.ops.make_list(
    a=weave.ops.TypedDict.merge(weave.ops.dict_(name='res0'), eval_result0_var['summary']),
    b=weave.ops.TypedDict.merge(weave.ops.dict_(name='res1'), eval_result1_var['summary']),
))

weave.ops.make_list(a=eval_result0_var['eval_table'], b=eval_result0_var['eval_table'])

# join evals together first
joined_evals = varbar.add('joined_evals', weave.ops.join_all(
    weave.ops.make_list(a=eval_result0_var['eval_table'], b=eval_result1_var['eval_table']),
    lambda row: row['dataset_id'],
    False))

# then join dataset to evals
dataset_evals = varbar.add('dataset_evals', weave.ops.join_2(
    dataset_var,
    joined_evals,
    lambda row: row['id'],
    lambda row: row['dataset_id'][0],
    'dataset',
    'evals',
    False,
    False
))


main = weave.panels.Group(
        layoutMode="grid",
        showExpressions=True,
        enableAddPanel=True,
    )

#### Run/config info TODO

#### Summary info

main.add("accuracy",
         weave.panels.Plot(summary, x=lambda row: row['accuracy'], y=lambda row: row['name']),
         layout=weave.panels.GroupPanelLayout(x=0, y=0, w=12, h=4))
# main.add('dataset_table', dataset)
# main.add('joined_evals', joined_evals)
# main.add('dataset_evals', dataset_evals, layout=weave.panels.GroupPanelLayout(x=0, y=4, w=24, h=6))

##### Example details

# more ideas: show examples that all got wrong, or that are confusing

eval_table = weave.panels.Table(dataset_evals)
eval_table.config.rowSize = 2
eval_table.add_column(lambda row: row['dataset.id'], 'id')
eval_table.add_column(lambda row: row['dataset.example'], 'example')
eval_table.add_column(lambda row: row['dataset.label'], 'label')
eval_table.add_column(lambda row: weave.ops.dict_(res0=row['evals.result'][0], res1=row['evals.result'][1]), 'result')
eval_table.add_column(lambda row: weave.ops.dict_(res0=row['evals.summary'][0], res1=row['evals.summary'][1]), 'summary')

main.add('eval_table', eval_table, layout=weave.panels.GroupPanelLayout(x=0, y=4, w=24, h=12))

weave.panels.Board(vars=varbar, panels=main)